# LSTM Based model for sequence tagging. 

## The data representation for the LSTM is done through enumeration (Index value of the Data). Therefore, when we provide new data, the enumerated list will break if new words are identified in the new dataset due this reason, the implementation of  predict_on_test_set() and predicts_on_text() cannot be done like I was able to do using the CRF method. 

## The alternative is to save the Word and Label list from the training and if new words are identified in the new test set, the word index values can be set to zero/null to proceed with predictions.


In [68]:
import pandas as pd
import csv
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.externals import joblib

from keras.preprocessing.sequence import pad_sequences
from keras.models import Model, Input
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from keras.utils import to_categorical

In [69]:
# Get sentences from data frame (CSV file)
def getSentences(df):
    agg_func = lambda s: [(word, pos, sp_tag, label) for word, pos, sp_tag, label in
                          zip(s["Word"].values.tolist(),s["POS"].values.tolist(),
                          s["Tag"].values.tolist(),s["Label"].values.tolist())]

    grouped = df.groupby("Sentence_Number").apply(agg_func)
    sentences = [s for s in grouped]

    return sentences

In [97]:
# Convert txt file to csv and use the CSV File. 
df = pd.read_csv('lstm_ner_dataset.csv', error_bad_lines=False)

b'Skipping line 93: expected 5 fields, saw 7\nSkipping line 150: expected 5 fields, saw 7\nSkipping line 216: expected 5 fields, saw 7\nSkipping line 221: expected 5 fields, saw 7\nSkipping line 310: expected 5 fields, saw 7\nSkipping line 347: expected 5 fields, saw 7\nSkipping line 387: expected 5 fields, saw 7\nSkipping line 442: expected 5 fields, saw 6\nSkipping line 448: expected 5 fields, saw 7\nSkipping line 458: expected 5 fields, saw 6\nSkipping line 463: expected 5 fields, saw 7\nSkipping line 480: expected 5 fields, saw 6\nSkipping line 503: expected 5 fields, saw 6\nSkipping line 523: expected 5 fields, saw 6\nSkipping line 527: expected 5 fields, saw 6\nSkipping line 539: expected 5 fields, saw 7\nSkipping line 577: expected 5 fields, saw 7\nSkipping line 606: expected 5 fields, saw 7\nSkipping line 634: expected 5 fields, saw 7\nSkipping line 642: expected 5 fields, saw 6\nSkipping line 646: expected 5 fields, saw 6\nSkipping line 726: expected 5 fields, saw 7\nSkipping 

In [72]:
# Get Unique Words in the dataset.
words = list(set(df["Word"].values))
words.append("PADDING")

In [73]:
# Number of words
number_of_words = len(words)

In [74]:
# Get Unique Labels in the dataset.
labels = list(set(df["Label"].values))

In [75]:
# Number of labels
number_of_lables = len(labels)

In [76]:
sentences = getSentences(df)

In [77]:
# Set maximum sentence length. If sentence length is less than max_sentence_length, add padding to the end.
max_sentence_length = 50

In [78]:
# Convert words into index values.
word2idx = {wordIdx: wordIndex for wordIndex, wordIdx in enumerate(words)}

In [79]:
# Convert labels into index values.
label2idx = {labelIdx: labelIndex for labelIndex, labelIdx in enumerate(labels)}

In [81]:
X = [[word2idx[w[0]] for w in s] for s in sentences]

In [82]:
# Pad sentences using pad_sequences method from Keras.
X = pad_sequences(maxlen=max_sentence_length, sequences=X, padding="post", value=number_of_words - 1)

In [83]:
# Read all the labels into X array.
y = [[label2idx[w[3]] for w in s] for s in sentences]

In [84]:
# Pad sentences using pad_sequences method from Keras.
y = pad_sequences(maxlen=max_sentence_length, sequences=y, padding="post", value=label2idx["O"])

In [85]:
# Convert Lables into Categorical values
y = [to_categorical(i, num_classes=number_of_lables) for i in y]

In [86]:
# Train Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

In [87]:
print('Number of Train Samples : ', len(X_train))
print('Number of Test Samples : ', len(X_test))

Number of Train Samples :  16596
Number of Test Samples :  1844


In [88]:
# Keras model creation
input = Input(shape=(max_sentence_length,))
model = Embedding(input_dim=number_of_words, output_dim=50, input_length=max_sentence_length)(input)
model = Dropout(0.1)(model)
model = Bidirectional(LSTM(units=100, return_sequences=True, recurrent_dropout=0.1))(model)
out = TimeDistributed(Dense(number_of_lables, activation="softmax"))(model)  # softmax output layer

In [89]:
model = Model(input, out)

In [90]:
model.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"])

In [91]:
model.fit(X_train, np.array(y_train), batch_size=32, epochs=2, validation_split=0.1, verbose=1)

Train on 14936 samples, validate on 1660 samples
Epoch 1/2
14936/14936 [==============================] - 46s 3ms/step - loss: 0.1591 - acc: 0.9605 - val_loss: 0.0913 - val_acc: 0.9707
Epoch 2/2
14936/14936 [==============================] - 41s 3ms/step - loss: 0.0642 - acc: 0.9807 - val_loss: 0.0527 - val_acc: 0.9851


In [93]:
joblib.dump(model, 'LSTM_Model') # Save the model

['LSTM_Model']

In [94]:
LSTM_Model_Name = 'LSTM_Model'

In [95]:
lstm = joblib.load(LSTM_Model_Name) # Load model

In [96]:
# Display results for all the test set sentences.
for test_sentence in X_test:
    p = lstm.predict(np.array([test_sentence]))
    p = np.argmax(p, axis=-1)
    for w, pred in zip(test_sentence, p[0]):
        print(words[w], labels[pred])
    print('\n')

In O
New B-LOC
York I-ORG
Steve B-PER
Finley I-PER
's O
three-run O
homer O
capped O
a O
four-run O
eighth O
inning O
and O
gave O
the O
San B-LOC
Diego I-ORG
Padres I-ORG
a O
4-3 O
victory O
over O
New B-ORG
York I-ORG
spoiling I-ORG
Bobby I-ORG
Valentine I-ORG
's O
debut O
as O
Mets B-ORG
' O
manager O
. O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O


" O
For O
the O
most O
part O
they O
really O
did O
n't O
understand O
what O
was O
going O
on O
" O
the O
director O
of O
the O
Nagyatad B-ORG
camp O
Lajos B-PER
Horvath I-PER
told O
Reuters B-ORG
on O
Wednesday O
. O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O


Current O
prft O
7 O
mln O
loss O
371 O
mln O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
P

BASKETBALL O
- O
PHILIPPINE B-PER
PRO-LEAGUE O
RESULTS O
. O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O


BAGHDAD B-LOC
1996-08-27 O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O


SOCCER O
- O
ROMANIA B-MISC
BEAT O
LITHUANIA O
IN O
WORLD O
CUP I-MISC
QUALIFIER O
. O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O


It O
was O
their O
fourth O
meeting O
this O
year O
. O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O


Death O
toll O
of O
Algeria B-LOC

CENTRAL B-MISC
DIVISION O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O


-DOCSTART- O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O

" O
The O
problem O
is O
over O
. O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O


Brann B-ORG
19 O
8 O
5 O
6 O
40 O
37 O
29 O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O


-DOCSTART- O
PADDING O
PADDING O
PADDING O
PADDING O
PAD

...

5. O
Bekescsaba B-ORG
3 O
2 O
- O
1 O
6 O
2 O
6 O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O


Ssangbangwool B-ORG
60 O
2 O
49 O
.550 O
5 O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O


LOS B-ORG
ANGELES I-ORG
71 O
61 O
.538 O
2 O
PADDIN

Local O
authorities O
launched O
an O
investigation O
after O
they O
received O
reports O
of O
several O
similar O
deaths O
in O
the O
area O
it O
said O
. O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O


-DOCSTART- O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O


minutes O
31.081 O
second

Revered O
skull O
of O
S. B-LOC
Africa I-LOC
king O
is O
Scottish B-MISC
woman O
's O
. O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O


Total O
( O
for O
one O
wicket O
) O
229 O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O


Port B-LOC
conditions O
from O
Lloyds B-ORG

1. O
Joel B-PER
Smets I-PER
( O
Belgium B-LOC
) O
Husaberg O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O


He O
said O
Talbott B-PER
who O
was O
scheduled O
to O
return O
on O
Tuesday O
would O
also O
to O
meet O
his O
Canadian B-MISC
counterpart O
Gordon B-PER
Smith I-PER
in O
Ottawa B-PER
for O
talks O
that O
would O
include O
the O
situation O
in O
Haiti O
. O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O


The O
former O
Wimbledon B-MISC
champion O
said O
the O


NEC B-ORG
Nijmegen I-ORG
1 O
( O
Van B-PER
Eykeren I-PER
15th O
) O
PSV B-ORG
Eindhoven I-ORG
4 O
( O
Numan O
11th O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O


TENNIS O
- O
RESULTS O
AT O
HAMLET B-MISC
CUP I-MISC
. O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O
PADDING O


SEATTLE B-ORG
9 O
Baltimore B-OR